In [57]:
import pandas as pd 
import os

## Посмотрим на наши файлы и сделаем из них csv

In [62]:
def convert_xlsx_to_csv(directory):
    files = os.listdir(directory)
    
    for file in files:
        if file.endswith('.xlsx'):
            file_path = os.path.join(directory, file)
            read_file = pd.read_excel(file_path, engine='openpyxl')
            csv_file_path = os.path.join(directory, file.replace('.xlsx', '.csv'))
            read_file.to_csv(csv_file_path, index=False, header=True)
            os.remove(file_path)

In [65]:
convert_xlsx_to_csv('data')

In [64]:
files = os.listdir('data')
file_names = [file.split('.')[0] for file in files]
file_paths = {file_name: file for file_name, file in zip(file_names, list(map(lambda x: 'data/' + x, files)))}
file_paths

{'brent_prices': 'data/brent_prices.csv',
 'eur_rate': 'data/eur_rate.csv',
 'gazp': 'data/gazp.csv',
 'sber': 'data/sber.csv',
 'usd_rate': 'data/usd_rate.csv'}

Ура, теперь все точно csv! Если изначально все файлы были в правильном формате, то функцию все равно можно вызывать, просто ничего не изменится.

## Обрабатываем данные по отдельности

### brent_prices

In [59]:
pd.read_csv('data/brent_prices.csv').head()

,Дата,Цена,Откр.,Макс.,Мин.,Объём,Изм. %
0,29.12.2023,"77,04","77,38","77,98","76,73","192,95K","-1,72%"
1,28.12.2023,"78,39","79,80","79,95","78,25","18,78K","-1,58%"
2,27.12.2023,"79,65","80,79","81,33","79,29","70,39K","-1,75%"
3,26.12.2023,"81,07","79,04","81,72","78,76","59,74K","2,53%"
4,22.12.2023,"79,07","79,35","80,37","78,88","119,19K","-0,40%"


In [60]:
# df = pd.read_csv('data/brent_prices.csv')
# df = df.rename(columns={'Дата': 'date', 'Цена': 'price', 'Откр.': 'open', 
#                         'Макс.': 'max', 'Мин.': 'min', 
#                         'Объём': 'volume', 'Изм. %': 'change'})
# df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
# df[['price', 'open', 'max', 'min']] = df[['price', 'open', 'max', 'min']].replace(',', '.', regex=True).astype(float)
# df['volume'] = df['volume'].replace('K', '', regex=True).replace(',', '.', regex=True).astype(float)
# df['change'] = df['change'].replace('%', '', regex=True).str.replace(',', '.', regex=True).astype(float)

# df.to_csv('data/brent_prices.csv', index=False)
# df

,date,price,open,max,min,volume,change
0,2023-12-29,77.04,77.38,77.98,76.73,192.95,-1.72
1,2023-12-28,78.39,79.80,79.95,78.25,18.78,-1.58
2,2023-12-27,79.65,80.79,81.33,79.29,70.39,-1.75
3,2023-12-26,81.07,79.04,81.72,78.76,59.74,2.53
4,2023-12-22,79.07,79.35,80.37,78.88,119.19,-0.40
...,...,...,...,...,...,...,...
1028,2020-01-08,65.44,71.22,71.75,64.94,560.89,-4.15
1029,2020-01-07,68.27,68.63,68.75,67.65,284.92,-0.93
1030,2020-01-06,68.91,69.35,70.74,68.21,341.14,0.45
1031,2020-01-03,68.60,66.28,69.50,66.22,479.70,3.55


### eur_rate

In [66]:
df = pd.read_csv('data/eur_rate.csv').head()
df

,nominal,data,curs,cdx
0,1,2023-12-30,89.6883,Доллар США
1,1,2023-12-29,90.3041,Доллар США
2,1,2023-12-28,91.7051,Доллар США
3,1,2023-12-27,91.7069,Доллар США
4,1,2023-12-26,91.9690,Доллар США


In [31]:
# df['nominal'].unique(), df['cdx'].unique()

(array([1], dtype=int64), array(['Доллар США'], dtype=object))

In [67]:
# df = df.drop(['nominal', 'cdx'], axis=1)
# df = df.rename(columns={'data': 'date'})
# df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
# df['curs'] = df['curs'].astype(float)

# df.to_csv('data/eur_rate.csv', index=False)
# df

,date,curs
0,2023-12-30,89.6883
1,2023-12-29,90.3041
2,2023-12-28,91.7051
3,2023-12-27,91.7069
4,2023-12-26,91.9690


### gazp

In [68]:
df = pd.read_csv('data/gazp.csv').head()
df

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,GAZP,D,200103,0,259.10,261.90,257.77,259.00,34326290
1,GAZP,D,200106,0,258.91,258.96,254.80,256.55,28106710
2,GAZP,D,200108,0,256.25,257.32,254.77,255.79,25566970
3,GAZP,D,200109,0,255.66,256.68,251.82,254.07,33194830
4,GAZP,D,200110,0,253.99,254.60,251.20,251.90,28545590


In [47]:
# df['<TICKER>'].nunique(), df['<PER>'].nunique(), df['<TIME>'].nunique()

In [69]:
# df = df.drop(['<TICKER>', '<PER>', '<TIME>'], axis=1)
# df = df.rename(columns={'<DATE>': 'date'})
# df['date'] = df['date'].apply(lambda x: '20' + str(x))
# df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
# df[[
#     '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>',
# ]] = df[[
#     '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>',
# ]].astype(float)

# df.to_csv('data/gazp.csv', index=False)
# df

,date,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,2020-01-03,259.10,261.90,257.77,259.00,34326290
1,2020-01-06,258.91,258.96,254.80,256.55,28106710
2,2020-01-08,256.25,257.32,254.77,255.79,25566970
3,2020-01-09,255.66,256.68,251.82,254.07,33194830
4,2020-01-10,253.99,254.60,251.20,251.90,28545590


### sber

In [77]:
df = pd.read_csv('data/sber.csv').head()
df

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,SBER,D,200103,0,255.97,258.19,253.70,255.00,35824160
1,SBER,D,200106,0,254.75,254.84,251.40,253.90,22327940
2,SBER,D,200108,0,253.62,259.15,253.03,259.15,40736030
3,SBER,D,200109,0,259.41,261.76,257.01,257.99,38196300
4,SBER,D,200110,0,257.91,259.25,256.88,258.19,18841650


In [78]:
# df = df.drop(['<TICKER>', '<PER>', '<TIME>'], axis=1)
# df = df.rename(columns={'<DATE>': 'date'})
# df['date'] = df['date'].apply(lambda x: '20' + str(x))
# df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
# df[[
#     '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>',
# ]] = df[[
#     '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>',
# ]].astype(float)

# df.to_csv('data/sber.csv', index=False)
# df

,date,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,2020-01-03,255.97,258.19,253.70,255.00,35824160
1,2020-01-06,254.75,254.84,251.40,253.90,22327940
2,2020-01-08,253.62,259.15,253.03,259.15,40736030
3,2020-01-09,259.41,261.76,257.01,257.99,38196300
4,2020-01-10,257.91,259.25,256.88,258.19,18841650


### usd_rate

In [81]:
df = pd.read_csv('data/usd_rate.csv').head()
df

,nominal,data,curs,cdx
0,1,2023-12-30,89.6883,Доллар США
1,1,2023-12-29,90.3041,Доллар США
2,1,2023-12-28,91.7051,Доллар США
3,1,2023-12-27,91.7069,Доллар США
4,1,2023-12-26,91.9690,Доллар США


In [82]:
# df = df.drop(['nominal', 'cdx'], axis=1)
# df = df.rename(columns={'data': 'date'})
# df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
# df['curs'] = df['curs'].astype(float)

# df.to_csv('data/usd_rate.csv', index=False)
# df

,date,curs
0,2023-12-30,89.6883
1,2023-12-29,90.3041
2,2023-12-28,91.7051
3,2023-12-27,91.7069
4,2023-12-26,91.9690


## Проверка, что никакие файлы не потерялись

In [83]:
os.listdir('data')

['brent_prices.csv',
 'eur_rate.csv',
 'gazp.csv',
 'sber.csv',
 'sber.csv, index=False',
 'usd_rate.csv',
 'usd_rate.csv, index=False']

In [84]:
for name, path in file_paths.items():
    print(name)
    print(pd.read_csv(path).columns)

brent_prices
Index(['date', 'price', 'open', 'max', 'min', 'volume', 'change'], dtype='object')
eur_rate
Index(['date', 'curs'], dtype='object')
gazp
Index(['date', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>'], dtype='object')
sber
Index(['date', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>'], dtype='object')
usd_rate
Index(['date', 'curs'], dtype='object')


Ура, эти 5 файлов немного высосали из меня силы, но зато теперь можно тестить второе задание и использовать этот ноутбук как пример для работы со следующими риск факторами.